In [ ]:
import numpy as np
from utils.defines import *
import os
import json

In [ ]:
# Load images, labels, predictions
images = []
labels = []
predictions = []
label_dict = {} # label: name of label, i.e., {0: dog, 1: cat}

In [ ]:
# The attribute you picked from prepare.py 
picked_attribute = []
types = []
attribute_dict = {}
main_object = ""

In [ ]:
# Define train_idx, valid_idx, unlabel_idx, data split
train_idx = []
valid_idx = []
split = [UNLABELED for i in range(len(images))]
for i in train_idx:
    split[i] = TRAIN
for i in valid_idx:
    split[i] = VALID

In [ ]:
# Build a base corpus
from utils.gpt import get_des_question, get_binary_question,get_des_prompt
from processData.Corpus import emptyAttribute
for name,data_type in zip(picked_attribute,types):
    attribute_dict[name] = emptyAttribute()
    attribute_dict[name]['type'] = data_type
    if data_type == 'description':
        attribute_dict[name]['question'] = get_des_question(main_object,name)
        attribute_dict[name]['prompt'] = [get_des_prompt(main_object,name),]
    else:
        attribute_dict[name]['question'] = get_binary_question(main_object,name)

# Although GPT is amazing, it is better to double-check the corpus base
for key, content in attribute_dict.items():
    print(key, content)

f = open("exampleData/corpus_base.json" ,'w')
f.write(json.dumps(attribute_dict,indent=4))

In [ ]:
# Find potential attribute value for this task
from processData.Corpus import AttributeCorpus
corpus = AttributeCorpus(datas=images,attribute_dict="exampleData/corpus_base.json")
corpus.auto_corpus()
corpus.export_json(file="exampleData/corpus.json")

In [ ]:
# Get the attribute of every data
from processData.AttributeSelection import AttributeSelection
AS = AttributeSelection(images, labels, corpus_path="exampleData/corpus.json",label_names_dict=label_dict)
AS.match_description_to_data()
AS.save_attributes("exampleData/attribute.json")

In [ ]:
# User Interface, require 2D feature as input
from interaction.interaction import Interaction
embeddings = np.load("") # if not 2D, we will run TSNE automatically
interaction = Interaction(
   datas=images,
   labels=labels,
   predictions=predictions,
   embeddings=embeddings,
   tags=split,
   attributes_dict_path="exampleData/attribute.json"
)
interaction.interaction()
 

In [ ]:
# Diagnose 
from processData.Diagnose import ModelDiagnose
MD = ModelDiagnose(labels, predictions, split, "exampleData/attribute.json")
MD.detect_failure_by_label()
MD.detect_prediction_correlation()
MD.detect_failure_prediction_correlation()
MD.detect_failure_in_attribute_combination()

In [ ]:
# Repair
selected_unlabeled_idx = MD.unlabel_selection()
generation_attribute_dict = MD.date_generation()